<a href="https://colab.research.google.com/github/Josh0527/Uber-Stcok-Prediction-Model/blob/main/Stock_Prediction_Model_(Uber).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#import .csv file

import kagglehub

# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/world-stock-prices-daily-updating")

print("Path to dataset files:", path)

100%|██████████| 11.8M/11.8M [00:00<00:00, 58.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nelgiriyewithana/world-stock-prices-daily-updating/versions/247


In [ ]:
#offload path into DF

from google.colab import files
uploaded = files.upload()

Saving World-Stock-Prices-Dataset.csv to World-Stock-Prices-Dataset.csv


In [ ]:
df = pd.read_csv("World-Stock-Prices-Dataset.csv")

df.head() #read first 5 rows

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Capital Gains
0,2025-01-21 00:00:00-05:00,8.400000,8.540000,8.260000,8.440000,7491400.0,0.0,0.0,peloton,PTON,fitness,usa,NaN
1,2025-01-21 00:00:00-05:00,10.230000,10.280000,9.940000,9.980000,6427300.0,0.0,0.0,zoominfo,ZI,technology,usa,NaN
2,2025-01-21 00:00:00-05:00,126.019997,132.220001,125.660004,132.220001,72400.0,0.0,0.0,adidas,ADDYY,apparel,germany,NaN
3,2025-01-21 00:00:00-05:00,316.940002,319.700012,315.000000,317.950012,2778300.0,0.0,0.0,american express,AXP,finance,usa,NaN
4,2025-01-21 00:00:00-05:00,42.000000,42.400002,41.700001,41.700001,100.0,0.0,0.0,puma,PMMAF,apparel,germany,NaN


In [ ]:
df.Brand_Name.unique()

array(['peloton', 'zoominfo', 'adidas', 'american express', 'puma',
       'visa', 'adobe', 'block', 'unilever', 'cisco',
       'jpmorgan chase & co', 'lvmh', 'airbnb', 'marriott', 'ubisoft',
       'toyota', 'amd', 'hilton', "mcdonald's", 'the home depot',
       'mastercard', 'johnson & johnson', 'uber', 'procter & gamble',
       'coinbase', 'fedex', '3m', 'nordstrom', 'philips', 'netflix',
       'the coca-cola company', 'foot locker', 'crocs',
       'southwest airlines', 'shopify', 'amazon', 'apple', 'nike',
       'target', 'google', 'spotify', 'zoom video communications',
       'the walt disney company', 'roblox', 'nintendo', 'delta air lines',
       'microsoft', 'costco', 'american eagle outfitters',
       'hershey company', 'tesla', 'pinterest', 'bmw group', 'chipotle',
       'porsche', 'logitech', 'colgate palmolive', 'salesforce / slack',
       'nvidia', 'starbucks', 'honda'], dtype=object)

In [ ]:
df1 = df[['Date', 'Close', 'Brand_Name']]
df1.head(2)

,Date,Close,Brand_Name
0,2025-01-21 00:00:00-05:00,8.44,peloton
1,2025-01-21 00:00:00-05:00,9.98,zoominfo


In [ ]:
df2 = df1[df1['Brand_Name'] == 'uber']

In [ ]:
df2.head(2), df2.shape

(                          Date      Close Brand_Name
 22   2025-01-21 00:00:00-05:00  67.739998       uber
 107  2025-01-17 00:00:00-05:00  67.339996       uber,
 (1470, 3))

In [ ]:
df3 = df2[['Date', 'Close']]
df3.tail(4)

,Date,Close
87714,2019-05-15 00:00:00-04:00,41.290001
87785,2019-05-14 00:00:00-04:00,39.959999
87847,2019-05-13 00:00:00-04:00,37.099998
87861,2019-05-10 00:00:00-04:00,41.570000


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1470 entries, 22 to 87861
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1470 non-null   object 
 1   Close   1470 non-null   float64
dtypes: float64(1), object(1)
memory usage: 34.5+ KB


In [ ]:
#covert to date dt
import datetime

def str_to_dt(s):
    split = s.split('-')
    year, month, day = int(split[0]), int(split[1]), int(split[2])
    return datetime.datetime(year=year, month=month, day=day)


In [ ]:
str_to_dt('2019-05-15 ')

Timestamp('2019-05-15 00:00:00')

In [ ]:
# Apply the function to the 'Date' column
df3['Date'] = df3['Date'].apply(str_to_dt)

NameError: name 'df3' is not defined